In [ ]:
from functools import partial
def compose(*workflow, expose):
    def dec(loss):
        @functools.wraps(loss)
        def called(**kwargs):
            
            for i,f in enumerate(workflow):
                
                sig = inspect.signature(f)
                f_args = sig.parameters.keys()
                #print([a for a in expose])
                
                arglist = []
                
#                 if len(expose) > 1:
                in_expose = False
                for arg in expose:
                    if arg in f_args:
                        #print(arg)
                        arglist.append(arg)
                        in_expose = True
                if in_expose:
                    #print(arglist)
                    f = partial(f, **{k:kwargs[k] for k in arglist})
#                 else:
#                     in_expose = False
#                     arg = expose[0]
#                     if arg in f_args:
#                         arglist = arg
#                         in_expose = True
#                     if in_expose:
#                         f = partial(f, **{arglist:kwargs[arglist]})
# #                 print(arglist, in_expose)
                
                if i == 0: 
                    res = f()
                else:
                    res = f(**res)
                    
            return loss(**res)
        
        return called
    
    return dec
            
    
            
def data_gen():
    return dict(data = [3,4])

def preprocess(data, params):
    s, b = data
    return dict(s = s + params, b = b)
    
@compose(data_gen, preprocess, expose = ['params'])
def loss(s,b):
    return s / b

loss(params=2)